In [1]:
import custom_utils as cu
from ultralytics import YOLO
import numpy as np

import cv2

labels = ["green", "red", "yellow"]

In [2]:
# prepare environment
import os
from dotenv import load_dotenv
# Load .env files
load_dotenv()

# Get environment variables
dataset_dir = os.getenv('DATASET_DIR')
ori_data_dir = os.path.join(dataset_dir, "traffic_light", "original_data")
custom_data_dir = os.path.join(dataset_dir, "traffic_light", "custom_data")

if not os.path.exists(ori_data_dir):
    print("Error: No original data set")

if not os.path.exists(custom_data_dir):
    os.mkdir(custom_data_dir)

for l in labels:
    label_dir = os.path.join(custom_data_dir, l)
    if not os.path.exists(label_dir):
        os.mkdir(label_dir)

In [14]:
# step 0: get images 
img_dic = {}
for l in labels: 
    paths, images = cu.get_images(os.path.join(ori_data_dir, l), 3)
    print(paths)
    print(os.path.join(ori_data_dir, l))
    img_dic.update({l: images })

/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/dataset/traffic_light/original_data/green
911
0 traffic-light--1-_jpg.rf.3ae134bdb27867d6c1a6a311e7eb8831.jpg
1 traffic-light--1-_jpg.rf.4a000eed21624b81d05ad71bc359e148.jpg
2 traffic-light--1-_jpg.rf.daef3a72f098a087aac1799a1c29ca89.jpg
3 traffic-light--10-_jpg.rf.322ccc6d4d307f5671d8edc784f02208.jpg
break
3
['/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/dataset/traffic_light/original_data/green/traffic-light--1-_jpg.rf.3ae134bdb27867d6c1a6a311e7eb8831.jpg', '/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/dataset/traffic_light/original_data/green/traffic-light--1-_jpg.rf.4a000eed21624b81d05ad71bc359e148.jpg', '/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/dataset/traffic_light/original_data/green/traffic-light--1-_jpg.rf.daef3a72f098a087aac1799a1c29ca89.jpg']
/home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/dataset/traffic_light/original_data/green
/home/jay/

In [15]:
# step 1: analyze images with yolo
model = YOLO("yolov8n.pt")
res_dict = {}

for l in labels:
    print(l)
    res = model.predict(img_dic[l], conf=0.3, save=True, save_txt = True)
    print(type(res))
    res_dict.update({l : res})

green



0: 640x640 1 apple, 1: 640x640 1 apple, 2: 640x640 (no detections), 454.5ms
Speed: 5.7ms preprocess, 151.5ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict30
2 labels saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict30/labels



<class 'list'>
red


0: 640x640 1 traffic light, 1: 640x640 1 traffic light, 2: 640x640 1 traffic light, 696.7ms
Speed: 5.3ms preprocess, 232.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict30
3 labels saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict30/labels



<class 'list'>
yellow


0: 640x640 1 traffic light, 1: 640x640 1 traffic light, 2: 640x640 1 traffic light, 586.0ms
Speed: 3.5ms preprocess, 195.3ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict30
3 labels saved to /home/jay/module/ai_app/self_driving_cars/aai-selfdriving-cars/runs/detect/predict30/labels


<class 'list'>


In [5]:
# todo 
# drop similar boxes is requiered becaus some times yolo identify the same object with a different bounding box
"""
def drop_similar_boxes(input_list, threshold):
    def are_elements_similar(elem1, elem2):
        x1_diff = abs(elem1[0] - elem2[0])
        y1_diff = abs(elem1[1] - elem2[1])
        x2_diff = abs(elem1[2] - elem2[2])
        y2_diff = abs(elem1[3] - elem2[3])
        return x1_diff < threshold or y1_diff < threshold or x2_diff < threshold or y2_diff < threshold

    unique_elements = []
    for elem in input_list:
        is_similar = False
        for unique_elem in unique_elements:
            if are_elements_similar(elem, unique_elem):
                is_similar = True
                break
        if not is_similar:
            unique_elements.append(elem)
    return unique_elements
"""
"""
input_list = [[1.0, 2.0, 3.0, 4.0], [1.2, 2.2, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0], [1.1, 2.1, 3.1, 4.1]]
threshold = 0.2
filtered_list = drop_similar_boxes(input_list, threshold)
print(filtered_list)
"""

'\ninput_list = [[1.0, 2.0, 3.0, 4.0], [1.2, 2.2, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0], [1.1, 2.1, 3.1, 4.1]]\nthreshold = 0.2\nfiltered_list = drop_similar_boxes(input_list, threshold)\nprint(filtered_list)\n'

In [6]:
def average_resize(label, images):
    print("type of images: ")
    print(type(images[0]))
    dia_len = [(i.shape[1]**2 + i.shape[2])**0.5 for i in images]
    avr = int(np.average(dia_len))
    for idx, i in enumerate(images):
        print("hi")
        i = cv2.resize(i, (avr,avr))
        #cv2.imwrite(os.path.join(custom_data_dir, label, str(idx), ".jpg"), i) # not nessecary that label is in the name of image

In [21]:
print(res_dict["green"])

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58

In [16]:
# step 2: extract all traffic lights from all analyzed images and create for each an new image

# Process results list
image_name = 'traffic_light_green'
file_type = '.jpg'

counter = 0
extracted_dic = {}
for l in labels:
    print(l)
    ex_images = [] # is an list because probably there are more traffic_lights objects in a picture
    for j, result in enumerate(res_dict[l]):
        for i,c in enumerate(result.boxes.cls.numpy()):
            # find traffic light boxes

            if c == 9:
                points = result.boxes.xyxyn.numpy()[i]
                extracted_image = cu.extract_rectangle_from_image(images[j], points) 
                ex_images.append(extracted_image)
                print(extracted_image)
                # res_dict[l][j] = extracted #
                # print(type(res_dict[l][j]))
                counter += 1
    
    extracted_dic.update({l : ex_images})

"""
# resize imgaes
print(extracted_dic[l])
average_resize(l, extracted_dic[l])
out_path = os.path.join(custom_data_dir, l)
print(out_path)
cu.write_images(extracted_dic[l], out_path, main_name=l)
"""

green
red
[[[ 55 182 107]
  [ 49 184 106]
  [ 34 168  95]
  ...
  [ 82  99  85]
  [ 93 110  96]
  [ 99 116 102]]

 [[ 63 189 113]
  [ 60 195 115]
  [ 56 190 114]
  ...
  [ 53  75  56]
  [ 62  81  62]
  [ 69  85  68]]

 [[ 54 179 100]
  [ 52 183 102]
  [ 64 194 117]
  ...
  [ 70 102  77]
  [ 67  90  68]
  [ 68  84  66]]

 ...

 [[ 76 107 100]
  [ 72  93  84]
  [ 64  78  66]
  ...
  [ 99 117 100]
  [101 117 100]
  [101 117 100]]

 [[ 75 104  95]
  [ 70  89  80]
  [ 62  76  64]
  ...
  [ 98 117 100]
  [ 99 117 100]
  [ 99 117 100]]

 [[ 81 106  96]
  [ 74  91  80]
  [ 66  79  65]
  ...
  [ 98 117 100]
  [ 98 117 100]
  [ 99 117 100]]]
[[[ 92 121 195]
  [ 91 121 192]
  [ 89 121 192]
  ...
  [142 140 140]
  [158 155 141]
  [157 152 127]]

 [[105 121 197]
  [104 121 194]
  [100 119 192]
  ...
  [169 181 191]
  [192 203 200]
  [202 212 196]]

 [[115 121 198]
  [116 122 199]
  [115 123 200]
  ...
  [174 193 208]
  [193 210 213]
  [204 220 213]]

 ...

 [[ 83 102  59]
  [ 66  90  42]
  [ 78 108

'\n# resize imgaes\nprint(extracted_dic[l])\naverage_resize(l, extracted_dic[l])\nout_path = os.path.join(custom_data_dir, l)\nprint(out_path)\ncu.write_images(extracted_dic[l], out_path, main_name=l)\n'

# step 3: check images manually and label them